In [2]:
## Use data of a production machine, where each row contains informations about a particular part
## Ceate a data point
dp1 = {'partno': 100, 'maxtemp': 35, 'mintemp': 35, 'maxvibration': 12, 'asperity': 0.32}
dp2 = {'partno': 101, 'maxtemp': 46, 'mintemp': 35, 'maxvibration': 21, 'asperity': 0.34}
dp3 = {'partno': 130, 'maxtemp': 56, 'mintemp': 46, 'maxvibration': 3412, 'asperity': 12.42}
dp4 = {'partno': 131, 'maxtemp': 58, 'mintemp': 48, 'maxvibration': 3542, 'asperity': 13.43}

In [3]:
def predict(dp):
    if dp['maxvibration']>100:
        return 13
    else:
        return 0.33

In [4]:
predict(dp1)
#predict(dp2)
#predict(dp3)
#predict(dp4)

0.33

In [5]:
## Liear regression is to create a linear relationship between filds by using fild numbers and a rate 'W' 
w1 = 0.33
w2 = 0
w3 = 0
w4 = 13/3412

def mlPredict(dp):
    return w1+w2*dp['maxtemp']+w3*dp['mintemp']+w4*dp['maxvibration']

In [6]:
mlPredict(dp4)

13.825310668229777

In [7]:
# The code was removed by Watson Studio for sharing.

+---+---+---+-----------+--------------------+
|  x|  y|  z|      class|              source|
+---+---+---+-----------+--------------------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|
| 22| 51| 34|Brush_teeth|Accelerometer-201...|
| 20| 50| 35|Brush_teeth|Accelerometer-201...|
| 22| 52| 34|Brush_teeth|Accelerometer-201...|
| 22| 50| 34|Brush_teeth|Accelerometer-201...|
| 22| 51| 35|Brush_teeth|Accelerometer-201...|
| 21| 51| 33|Brush_teeth|Accelerometer-201...|
| 20| 50| 34|Brush_teeth|Accelerometer-201...|
| 21| 49| 33|Brush_teeth|Accelerometer-201...|
| 21| 49| 33|Brush_teeth|Accelerometer-201...|
| 20| 51| 35|Brush_teeth|Accelerometer-201...|
| 18| 49| 34|Brush_teeth|Accelerometer-201...|
| 19| 48| 34|Brush_teeth|Accelerometer-201...|
| 16| 53| 34|Brush_teeth|Accelerometer-201...|
| 18| 52| 35|

In [8]:
df.createOrReplaceTempView('df')

df_energy = spark.sql("""
    select sqrt(sum(x*x)+sum(y*y)+sum(z*z)) as label, class from df group by class
""")

df_energy.createOrReplaceTempView('df_energy')

In [9]:
df_join = spark.sql("""
    select * from df inner join df_energy on df.class=df_energy.class
""")

In [10]:
df_join.show()

+---+---+---+-----------+--------------------+-----------------+-----------+
|  x|  y|  z|      class|              source|            label|      class|
+---+---+---+-----------+--------------------+-----------------+-----------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 51| 34|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 20| 50| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 52| 34|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 50| 34|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 22| 51| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|

In [11]:
from pyspark.ml.feature import VectorAssembler, Normalizer
from pyspark.ml.linalg import Vectors

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

In [12]:
from pyspark.ml.regression import LinearRegression

In [13]:
Lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [14]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages =[vectorAssembler, normalizer, Lr])

model = pipeline.fit(df_join)


In [24]:
prediction = model.transform(df_join)
model.stages[2].summary.r2

0.09123304798488863

In [20]:
prediction.show()

+---+---+---+-----------+--------------------+-----------------+-----------+----------------+--------------------+------------------+
|  x|  y|  z|      class|              source|            label|      class|        features|       features_norm|        prediction|
+---+---+---+-----------+--------------------+-----------------+-----------+----------------+--------------------+------------------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[22.0,49.0,35.0]|[0.20754716981132...|11102.707575524197|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[22.0,49.0,35.0]|[0.20754716981132...|11102.707575524197|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[22.0,52.0,35.0]|[0.20183486238532...|11122.581426949906|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[22.0,52.0,35.0]|[0.20183486238532...|11122.581426949906|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|11785.3963446292